In [ ]:
#-----------------------------------------------------------------------Retrieve a lost Databricks SQL Query-------------------------------------------------------------------
# Get a sql query to recreate query, search json data to find query using the function get_all_files_from_s3(bucket, folder).
# For refrence, all jobs are backed up on Wednesday, use a Wednesday date.
# Files are saved accoring to user that created the query

In [ ]:
import requests
import json
import pandas as pd
from datetime import date
import boto3
import os
from urllib.parse import urljoin

In [ ]:
DBATSECRETSCOPE = "Production-ELT"
DBPATKEY = "DB_API_Token"

host_token = dbutils.secrets.get(DBATSECRETSCOPE, DBPATKEY)

# API key to authenticate with the Databricks REST API
headers = {"Authorization": "Bearer " + host_token}

# configure workspace URL
host_name = "<work-place-url>"

# accessing AWS S3 
bucket = '<workplace-s3-bucket>'
main_folder = 'databricks-queries-settings-weekly'
# Initialize the S3 client
s3 = boto3.client('s3')

In [ ]:
%run "/PROD/Reuseable_Code/nb_prod_functions_job_backup_rebuild"

In [ ]:
folder_path = f's3a://{bucket}/{main_folder}'
file_list = dbutils.fs.ls(folder_path)

for subfolder_name in file_list:
    print(file.name)

In [ ]:
subfolder_name = 'aletia_trepte/'
subfolder_path = f's3a://{bucket}/{main_folder}/{subfolder_name}'
subfolder_files_list = dbutils.fs.ls(subfolder_path)
n = 0
for subfolder_files in subfolder_files_list:
    file_path = subfolder_files.path
    df = spark.read.option("multiline", "true").json(file_path)
    queries = df.select("query").rdd.flatMap(lambda x: x)
    for query in queries.collect():
        print('**********************************************************')
        print(f'                       QUERY {n}                          ')
        print('**********************************************************')
        if query:  # Check if the query is not null
            for line in query.split('\n'):
                print(line)
            n = n+1

In [ ]:
subfolder_name = 'aletia_trepte/'
subfolder_path = f's3a://{bucket}/{main_folder}/{subfolder_name}'
subfolder_files_list = dbutils.fs.ls(subfolder_path)

# Variable to store the first query
first_query = None
n = 0

for subfolder_files in subfolder_files_list:
    file_path = subfolder_files.path
    df = spark.read.option("multiline", "true").json(file_path)
    queries = df.select("query").rdd.flatMap(lambda x: x)
    
    for query in queries.collect():
        if n == 0 and query:  # Save the first query
            first_query = query
            break
        n += 1
    
    if first_query is not None:
        break  # Exit the outer loop if the first query is found

# Print the first query if it exists
if first_query:
    print('**********************************************************')
    print(f'                       QUERY {n}                          ')
    print('**********************************************************')
    for line in first_query.split('\n'):
        print(line)
else:
    print("No query found.")


In [ ]:
result_df = spark.sql(first_query)
result_df.show()